In [25]:
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization

#from xception import Xception

In [26]:
def preprocess_input(x):
    x /= 255.0
    x -= 0.5
    x *= 2.0
    return x

In [27]:
train_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/train'
val_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/val'
#test_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/test'

In [28]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)))
model.add(BatchNormalization())
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

model.add(layers.Flatten())
#model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(layers.Dense(20, activation='softmax'))



#model = Xception(weight_decay=1e-5, classes=20)
model.count_params()

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 297, 297, 32)      896       
_________________________________________________________________
batch_normalization_10 (Batc (None, 297, 297, 32)      128       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 295, 295, 32)      9248      
_________________________________________________________________
batch_normalization_11 (Batc (None, 295, 295, 32)      128       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 147, 147, 32)      0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 147, 147, 32)      0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 145, 145, 64)     

In [29]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)
'''
data_generator = ImageDataGenerator(
    rotation_range=30, 
    zoom_range=0.3,
    horizontal_flip=True, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.001,
    channel_shift_range=0.1,
    fill_mode='reflect',
    data_format='channels_last',
    preprocessing_function=preprocess_input
)
#val_datagen = ImageDataGenerator(rescale=1./255)
#test_datagen = ImageDataGenerator(rescale=1./255)
'''
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    shuffle=False)

validation_generator = data_generator.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=32,
    shuffle=False)

'''
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=20,
    shuffle=False)
'''

Found 2000 images belonging to 20 classes.
Found 400 images belonging to 20 classes.


'\ntest_generator = data_generator.flow_from_directory(\n    test_dir,\n    target_size=(299, 299),\n    batch_size=20,\n    shuffle=False)\n'

# Model Compilation

In [30]:
from keras.losses import categorical_crossentropy as logloss
from keras import backend as K

def loss(y_true, y_pred):
    entropy = -K.mean(K.sum(y_pred*K.log(y_pred), 1))
    beta = 0.1
    return logloss(y_true, y_pred) - beta*entropy

In [31]:
from keras import optimizers

model.compile(loss=loss,
    optimizer=optimizers.SGD(lr=1e-2),
    metrics=['categorical_crossentropy', 'accuracy', 'top_k_categorical_accuracy'])

# Training

In [32]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=118,
    epochs=10,
    validation_data=validation_generator, validation_steps=30)

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[32,295,295,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node SquaredDifference_10-0-0-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_23607]

Function call stack:
keras_scratch_graph


In [ ]:
model.save_weights('teacher_weights.hdf5')